In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split, DataLoader, Dataset

import torch
from torchvision import datasets, transforms

In [ ]:
%pip install -q transformers

Найдем все json файлы с аннотациями

In [5]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

Делаем кроп и сохраняем файлы и аннотации с номерами играков

In [9]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    x = pd.read_json(anno, orient='index')
    frame = x.iloc[3, :][0]

    img_path = anno.replace('anno', 'frames')
    img_path = img_path.replace('.json', '.jpeg')
    img = Image.open(img_path)

    for j in range(len(points)):
        out_file = {}
        point_obj = pd.DataFrame(points.iloc[j, 0:2])
        point = point_obj.iloc[1, :]
        label = point_obj.iloc[0, :]
        split_label = label[j].split('_')
        sp = split_label[-1]

        for box in point:   

            if sp.isdigit():

                xtl, ytl, xbr, ybr = int(box[0][0]), int(box[0][1]), int(box[1][0]), int(box[1][1])
                number = sp
                print(xtl, ytl, xbr, ybr)

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                print(x_dif, y_dif)
                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                print(xtl, ytl, xbr, ybr)
                break

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'i{str(num)}example.jpg'
                num += 1

                annos_list.append(str(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_0.csv")
df.head()
                

   

  0%|          | 25/5598 [00:00<00:46, 120.41it/s]

1559 700 1576 714
17 14
1558.15 699.3 1576.85 714.7
1171 654 1184 664
13 10
1170.35 653.5 1184.65 664.5
1572 692 1586 704
14 12
1571.3 691.4 1586.7 704.6
1201 661 1216 672
15 11
1200.25 660.45 1216.75 672.55
1579 694 1596 707
17 13
1578.15 693.35 1596.85 707.65
1232 667 1246 678
14 11
1231.3 666.45 1246.7 678.55
1594 690 1609 703
15 13
1593.25 689.35 1609.75 703.65
1257 665 1273 677
16 12
1256.2 664.4 1273.8 677.6
1284 673 1300 685
16 12
1283.2 672.4 1300.8 685.6
1225 760 1248 790
23 30
1223.85 758.5 1249.15 791.5
1287 745 1308 783
21 38
1285.95 743.1 1309.05 784.9
1407 636 1415 644
8 8
1406.6 635.6 1415.4 644.4
1343 749 1365 776
22 27
1341.9 747.65 1366.1 777.35
1402 750 1422 781
20 31
1401.0 748.45 1423.0 782.55
1446 729 1461 755
15 26
1445.25 727.7 1461.75 756.3
1399 631 1406 638
7 7
1398.65 630.65 1406.35 638.35
1479 715 1500 743
21 28
1477.95 713.6 1501.05 744.4
1400 625 1408 633
8 8
1399.6 624.6 1408.4 633.4
1502 723 1524 748
22 25
1500.9 721.75 1525.1 749.25
1164 624 1175 638
11

  1%|          | 38/5598 [00:00<00:45, 121.44it/s]

1411 621 1420 636
9 15
1410.55 620.25 1420.45 636.75
1406 621 1418 638
12 17
1405.4 620.15 1418.6 638.85
1401 621 1416 639
15 18
1400.25 620.1 1416.75 639.9
1395 616 1411 637
16 21
1394.2 614.95 1411.8 638.05
1390 609 1400 623
10 14
1389.5 608.3 1400.5 623.7
1384 604 1397 622
13 18
1383.35 603.1 1397.65 622.9
1384 601 1395 617
11 16
1383.45 600.2 1395.55 617.8
1383 599 1394 614
11 15
1382.45 598.25 1394.55 614.75
1384 598 1395 615
11 17
1383.45 597.15 1395.55 615.85
1120 647 1135 662
15 15
1119.25 646.25 1135.75 662.75
1108 644 1124 657
16 13
1107.2 643.35 1124.8 657.65
1383 593 1391 608
8 15
1382.6 592.25 1391.4 608.75
1107 636 1121 649
14 13
1106.3 635.35 1121.7 649.65
1382 590 1391 603
9 13
1381.55 589.35 1391.45 603.65
1115 627 1128 643
13 16
1114.35 626.2 1128.65 643.8
1388 595 1393 606
5 11
1387.75 594.45 1393.25 606.55
1123 631 1136 647
13 16
1122.35 630.2 1136.65 647.8
1387 593 1394 604
7 11
1386.65 592.45 1394.35 604.55
1140 625 1155 640
15 15
1139.25 624.25 1155.75 640.75
138

  1%|▏         | 80/5598 [00:00<00:42, 129.79it/s]

...

 16%|█▌        | 888/5598 [00:07<00:35, 132.56it/s]

1494 603 1518 644
24 41
1492.8 600.95 1519.2 646.05
1473 599 1494 635
21 36
1471.95 597.2 1495.05 636.8
1453 601 1477 637
24 36
1451.8 599.2 1478.2 638.8
1396 597 1414 633
18 36
1395.1 595.2 1414.9 634.8
1345 592 1368 622
23 30
1343.85 590.5 1369.15 623.5
1286 586 1309 618
23 32
1284.85 584.4 1310.15 619.6
1273 581 1286 610
13 29
1272.35 579.55 1286.65 611.45
1246 587 1270 618
24 31
1244.8 585.45 1271.2 619.55


 16%|█▌        | 896/5598 [00:07<00:38, 120.93it/s]


405 581 426 615
21 34
403.95 579.3 427.05 616.7
234 583 259 609
25 26
232.75 581.7 260.25 610.3


KeyboardInterrupt: 

Найдем все папки в которых лежит вайл с аннотацией

In [72]:

crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))      
     

In [74]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.4

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        file_name = 'data\\' + file_name
  
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + str(num) + 'example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno.csv")
df.head()


100%|██████████| 105/105 [19:21<00:00, 11.07s/it]


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,8
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,11
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3


In [ ]:
df = pd.read_csv('anno.csv')
df.head()

In [75]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [76]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
        #print(type(text))
        # prepare image (i.e. resize + normalize)
        image = Image.open(file_name).convert("RGB")
        image = image.resize((64, 64))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [77]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [78]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests

#processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [79]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 21087
Number of validation examples: 5272


In [80]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### В этом блоке мы смотрим в ручном режиме как решает нашу задачу модель из коробки

In [81]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Номер на футболке: 35


In [22]:
transform = transforms.Compose(
        [transforms.RandomResizedCrop(224), 
         transforms.ToTensor()])

In [82]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

In [83]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=3,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="./",
    logging_steps=50,
    save_steps=10000,
    eval_steps=2000,
)

In [84]:
from datasets import load_metric

cer_metric = load_metric("cer")

C:\Users\Mytre\AppData\Local\Temp\ipykernel_28632\152175726.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [85]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [86]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.01, no_deprecation_warning=True)


In [87]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/21087 [00:00<?, ?it/s]

{'loss': 2.9463, 'learning_rate': 4.988618580167877e-05, 'epoch': 0.01}
{'loss': 1.8667, 'learning_rate': 4.976762934509414e-05, 'epoch': 0.01}
{'loss': 1.8245, 'learning_rate': 4.964907288850951e-05, 'epoch': 0.02}
{'loss': 1.7165, 'learning_rate': 4.953051643192488e-05, 'epoch': 0.03}
{'loss': 1.5745, 'learning_rate': 4.941195997534026e-05, 'epoch': 0.04}
{'loss': 1.4029, 'learning_rate': 4.929340351875563e-05, 'epoch': 0.04}
{'loss': 1.2278, 'learning_rate': 4.917484706217101e-05, 'epoch': 0.05}
{'loss': 1.2208, 'learning_rate': 4.905629060558638e-05, 'epoch': 0.06}
{'loss': 1.2329, 'learning_rate': 4.893773414900176e-05, 'epoch': 0.06}
{'loss': 1.0937, 'learning_rate': 4.881917769241713e-05, 'epoch': 0.07}
{'loss': 1.152, 'learning_rate': 4.87006212358325e-05, 'epoch': 0.08}
{'loss': 0.9423, 'learning_rate': 4.858206477924788e-05, 'epoch': 0.09}
{'loss': 1.0039, 'learning_rate': 4.846350832266325e-05, 'epoch': 0.09}
{'loss': 1.0023, 'learning_rate': 4.834495186607863e-05, 'epoch': 

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


  0%|          | 0/1758 [00:00<?, ?it/s]

{'eval_loss': 1.4259974956512451, 'eval_cer': 0.9032098153426695, 'eval_runtime': 1018.6283, 'eval_samples_per_second': 5.176, 'eval_steps_per_second': 1.726, 'epoch': 0.28}
{'loss': 1.3998, 'learning_rate': 4.514629866742543e-05, 'epoch': 0.29}
{'loss': 1.2838, 'learning_rate': 4.502774221084081e-05, 'epoch': 0.3}
{'loss': 1.2947, 'learning_rate': 4.490918575425618e-05, 'epoch': 0.31}
{'loss': 1.269, 'learning_rate': 4.479062929767155e-05, 'epoch': 0.31}
{'loss': 1.2084, 'learning_rate': 4.4672072841086924e-05, 'epoch': 0.32}
{'loss': 1.2192, 'learning_rate': 4.45535163845023e-05, 'epoch': 0.33}
{'loss': 1.1216, 'learning_rate': 4.4434959927917674e-05, 'epoch': 0.33}
{'loss': 1.0528, 'learning_rate': 4.431640347133305e-05, 'epoch': 0.34}
{'loss': 0.9805, 'learning_rate': 4.419784701474843e-05, 'epoch': 0.35}
{'loss': 0.9069, 'learning_rate': 4.40792905581638e-05, 'epoch': 0.36}
{'loss': 0.9572, 'learning_rate': 4.396073410157917e-05, 'epoch': 0.36}
{'loss': 0.9627, 'learning_rate': 4.

  0%|          | 0/1758 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [91]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (4) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Номер на футболке: 55
